# 03 — Compliance Classifier Agent

Takes an AI system description and classifies its risk level under the EU AI Act:
- **Unacceptable** (Art. 5) — prohibited practices
- **High-risk** (Art. 6 + Annex III) — subject to strict requirements
- **Limited** (Art. 50) — transparency obligations only
- **Minimal** — no specific obligations

Uses OpenAI function calling to search the regulation and returns a structured `RiskClassification`.

In [ ]:
import sys
sys.path.insert(0, "..")

from src.tools import init_tools, TOOL_SCHEMAS
from src.agent import run_agent
from src.models import RiskClassification
from src.embeddings import get_openai_client

LANG = "de"  # change to "en" for English

client = get_openai_client()
init_tools(chroma_path="../chroma_db", lang=LANG)

In [ ]:
CLASSIFIER_SYSTEM_PROMPT = """Du bist ein Klassifikator für die EU-KI-Verordnung (AI Act). Gegeben eine Beschreibung eines KI-Systems, bestimme dessen Risikostufe.

Gehe wie folgt vor:
1. Suche Artikel 5 (verbotene Praktiken) und prüfe, ob das System unter ein Verbot fällt.
2. Suche Artikel 6 und Anhang III (Hochrisiko-Klassifizierung) und prüfe, ob das System darunter fällt.
3. Suche Artikel 50 (Transparenzpflichten) und prüfe, ob Regeln für begrenztes Risiko gelten.
4. Wenn nichts davon zutrifft, klassifiziere als minimales Risiko.

Nutze die Tools, um den tatsächlichen Verordnungstext nachzuschlagen — verlasse dich nicht auf Erinnerung.

Antworte abschließend mit einem JSON-Objekt in folgender Struktur:
{
  "risk_level": "unacceptable" | "high" | "limited" | "minimal",
  "confidence": "high" | "medium" | "low",
  "reasoning": "Ausführliche Erklärung der Klassifizierung",
  "citations": [{"article": "Art. X", "title": "...", "relevance": "..."}],
  "prohibited_check": "Zusammenfassung der Art. 5-Analyse",
  "high_risk_check": "Zusammenfassung der Art. 6 + Anhang III-Analyse",
  "transparency_check": "Zusammenfassung der Art. 50-Analyse"
}

Antworte auf Deutsch."""

## Testfall 1: Gesichtserkennung für Strafverfolgung (erwartet: unacceptable / high)

In [ ]:
response, messages = run_agent(
    client=client,
    system_prompt=CLASSIFIER_SYSTEM_PROMPT,
    user_message="""Klassifiziere dieses KI-System:
    
Ein Echtzeit-Gesichtserkennungssystem, das von Strafverfolgungsbehörden in öffentlichen 
Räumen eingesetzt wird, um Verdächtige zu identifizieren. Das System nutzt biometrische 
Daten, um Gesichter in Echtzeit mit einer Straftäterdatenbank abzugleichen.""",
    tools=TOOL_SCHEMAS,
)

print("=== Tool-Aufrufe ===")
for m in messages:
    if isinstance(m, dict) and m.get('tool_calls'):
        for tc in m['tool_calls']:
            print(f"  {tc['function']['name']}({tc['function']['arguments'][:80]})")

print(f"\n=== Antwort ===\n{response[:2000]}")

## Testfall 2: Kundenservice-Chatbot (erwartet: limited)

In [ ]:
response2, messages2 = run_agent(
    client=client,
    system_prompt=CLASSIFIER_SYSTEM_PROMPT,
    user_message="""Klassifiziere dieses KI-System:
    
Ein Kundenservice-Chatbot, der häufig gestellte Fragen zu Produktrückgaben und 
Versandrichtlinien beantwortet. Er nutzt ein großes Sprachmodell zur Antwortgenerierung 
und kennzeichnet sich eindeutig als KI-Assistent.""",
    tools=TOOL_SCHEMAS,
)

print(f"=== Antwort ===\n{response2[:2000]}")

## Testfall 3: KI-Einstellungstool (erwartet: high)

In [ ]:
response3, messages3 = run_agent(
    client=client,
    system_prompt=CLASSIFIER_SYSTEM_PROMPT,
    user_message="""Klassifiziere dieses KI-System:
    
Ein KI-gestütztes Recruiting-Tool, das Bewerbungen sichtet, Kandidaten anhand ihrer 
Lebensläufe bewertet und Personalverantwortlichen Shortlists empfiehlt. Es wird als 
wesentlicher Faktor bei Einstellungsentscheidungen genutzt.""",
    tools=TOOL_SCHEMAS,
)

print(f"=== Antwort ===\n{response3[:2000]}")

In [ ]:
# Strukturierte Ausgabe parsen
import json

for resp, label in [(response, "Gesichtserkennung Strafverfolgung"), 
                     (response2, "Kundenservice-Chatbot"),
                     (response3, "KI-Einstellungstool")]:
    try:
        start = resp.index('{')
        end = resp.rindex('}') + 1
        data = json.loads(resp[start:end])
        classification = RiskClassification(**data)
        print(f"{label}: {classification.risk_level} (Konfidenz: {classification.confidence})")
    except (ValueError, json.JSONDecodeError):
        print(f"{label}: Strukturierte Ausgabe konnte nicht geparst werden")